In [4]:
import numpy as np
from PIL import Image
import glob, os, sys, tqdm
import matplotlib.pyplot as plt 
import pandas as pd

path = '/data/mint/DPM_Dataset/Generated_Dataset_TargetLight/Generated_Dataset_cast_shadows_extra_rotate/'
sub_p = '/shadow_diff_SS_with_c_simplified/train/'

out_p = '/shadow_diff_SS_with_c_simplified_npy/train/'

os.makedirs(f'{path}/{out_p}', exist_ok=True)
all_imgs = glob.glob(f'{path}/{sub_p}/*.png')
use_c = True # False

# Load c values
c_p = '/data/mint/DPM_Dataset/ffhq_256_with_anno/params/train/ffhq-train-shadow-anno.txt'
c_dict = pd.read_csv(c_p, sep=' ', header=None, names=['image_name', 'c_val'])
max_c = 8.481700287326827 # 7.383497233314015
min_c = -4.989461058405101 # -4.985533880236826

for img_path in tqdm.tqdm(all_imgs):
    img_name = img_path.split('/')[-1]
    input_name = img_name.split('.png')[0]
    # print(input_name)
    if 'relit' in input_name:
        tmp = input_name.split('_relit')[0]
        # print(tmp)
        src = tmp.split('_')[0]
        dst = tmp.split('_')[1]
        c_val = c_dict[c_dict['image_name'] == f'{dst}.jpg']['c_val'].values[0]
    elif 'input' in input_name:
        tmp = input_name.split('_input')[0]
        src = tmp.split('_')[0]
        # dst = tmp.split('_')[1]
        c_val = c_dict[c_dict['image_name'] == f'{src}.jpg']['c_val'].values[0]
    
    c_val_norm = (c_val - min_c) / (max_c - min_c)
    
    img = Image.open(img_path)
    img = np.array(img)
    assert np.allclose(img[..., 0], img[..., 1]) and np.allclose(img[..., 1], img[..., 2])
    img = img[..., 0]
    # print("C val: ", c_val, c_val_norm)
    
    if use_c:
        shadows = img != 0
        out = shadows * c_val_norm
    else:
        out = img / 255.
        
    out = out.astype(np.float32)[..., None]
    np.save(f'{path}/{out_p}/{input_name}.npy', out)
    # print(img.shape, img.max(), img.min(), np.unique(img))
    # print(out.shape, out.max(), out.min(), np.unique(out))
    # plt.imshow(img)
    # plt.show()
    
    # assert False

100%|██████████| 10000/10000 [00:45<00:00, 221.91it/s]
